In [32]:
from google.colab import drive
drive.mount("/gdrive")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [33]:
!pip install mne

In [34]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import os
import mne

In [35]:
cd /gdrive/MyDrive/BCSDD

/gdrive/MyDrive/BCSDD


In [36]:
data = sio.loadmat('dataset.mat')

In [37]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Dec 12 16:15:31 2019',
 '__version__': '1.0',
 '__globals__': [],
 'EEGsample': array([[[ 4.08097622e+00,  5.36373657e+00,  2.16892286e+00, ...,
          -2.43467111e+00, -1.08033326e+01, -4.06570287e+00],
         [ 3.78602291e+00,  6.18806022e+00,  1.72710622e+00, ...,
          -2.23788269e+00, -1.08040545e+01, -3.16985550e+00],
         [ 8.34680437e+00,  8.69160090e+00, -1.09056499e-02, ...,
           2.30521997e+00, -1.33921422e+01, -5.27739673e+00],
         ...,
         [ 1.52884814e+00,  7.93795880e-01, -1.45126572e+00, ...,
          -1.22710349e+00, -5.26310889e+00, -1.22684549e+00],
         [-1.19949821e+00, -1.54496214e+00, -4.31985930e+00, ...,
          -5.32550694e+00, -5.48814272e+00, -1.57636907e+00],
         [ 1.59400945e+00,  1.90051435e+00, -4.64426331e+00, ...,
          -3.48251971e+00, -2.09547756e+00,  4.01980667e+00]],
 
        [[ 5.10435476e+00,  2.66309990e-01,  3.92736106e+00, ..

In [58]:
eeg_data= np.array(data['EEGsample'])
# srate= data['nfo']['fs'][0][0][0][0]

chan_names= ["Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8", "FT7", "FC3", "FCZ", "FC4", "FT8", "T3", "C3", "Cz", "C4", "T4", "TP7", "CP3", "CPz", "CP4", "TP8", "T5", "P3", "PZ", "P4", "T6", "O1", "Oz" ,"O2",
 "0",    "1",  "2",  "3",  "4",  "5",  "6",  "7",   "8",   "9",   "10",   "11", "12", "13", "14", "15", "16", "17",  "18",  "19",  "20",  "21",  "22",  "23","24", "25", "26", "27", "28", "29"]
# nchannels,nsamples=eeg_data.shape

labels =np.array(data["substate"])
labels.astype(int)
samplenum=labels.shape[0]

# event_onsets= data['mrk'][0][0][0]
# event_codes= data['mrk'][0][0][1]

# labels=np.zeros((1,nsamples),dtype=int)
# labels[0,event_onsets]=event_codes

# cl_lab=[s[0] for s in data['nfo']['classes'][0][0][0]]
# cl1=cl_lab[0]
# cl2=cl_lab[1]

# xpos=data['nfo']['xpos'][0][0]
# ypos=data['nfo']['ypos'][0][0]

# nclasses=len(cl_lab)
# nevents=len(event_onsets)

#   only channel 28 is used, which corresponds to the Oz channel
selectedchan=[28]

#   update the xdata and channel number
xdata=eeg_data[:,selectedchan,:]
channelnum=len(selectedchan)
channelnum

1

In [39]:
len(xdata)

2022

In [40]:
#   there are 11 subjects in the dataset. Each sample is 3-seconds data from 30 channels with sampling rate of 128Hz.
channelnum=30
subjnum=11
samplelength=3
sf=128

#   define the learning rate, batch size and epoches
lr=1e-2
batch_size = 50
n_epoch =6

In [64]:
model = models.Sequential([
    layers.Conv2D(filters=32,kernel_size=(1,64),input_shape=(1,1,384)),
    layers.BatchNormalization(),
    layers.ELU(alpha=1),
    layers.AveragePooling2D(pool_size=(1,321)),
    layers.Reshape((-1,)),
    layers.Dense(2,input_dim=32),
    layers.Softmax()

])

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_8. Consider increasing the input size. Received input shape [None, 1, 1, 384] which would produce output shape with a zero or negative value in a dimension.

In [53]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [54]:
results=np.zeros(subjnum)

In [56]:
subIdx = np.array(data['subindex'])
ydata = labels

In [65]:
for i in range(1,subjnum+1):

#       form the training data
        trainindx=np.where(subIdx != i)[0]
        xtrain=xdata[trainindx]
        x_train = xtrain.reshape(xtrain.shape[0],1,channelnum, samplelength*sf)
        y_train=ydata[trainindx]


#       form the testing data
        testindx=np.where(subIdx == i)[0]
        xtest=xdata[testindx]
        x_test = xtest.reshape(xtest.shape[0], 1,channelnum, samplelength*sf)
        y_test=ydata[testindx]


#         train = torch.utils.data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
#         train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

# #       load the CNN model to deal with 1D EEG signals
#         my_net = CompactCNN().double().cuda()


        # optimizer = optim.Adam(my_net.parameters(), lr=lr)
        # loss_class = torch.nn.NLLLoss().cuda()

        # for p in my_net.parameters():
        #     p.requires_grad = True


        model.fit(x_train,y_train,epochs=10)

#        train the classifier
        # for epoch in range(n_epoch):
        #     for j, data in enumerate(train_loader, 0):
        #         inputs, labels = data

        #         input_data = inputs.cuda()
        #         class_label = labels.cuda()

        #         my_net.zero_grad()
        #         my_net.train()

        #         class_output= my_net(input_data)
        #         err_s_label = loss_class(class_output, class_label)
        #         err = err_s_label

        #         err.backward()
        #         optimizer.step()

#       test the results
        model.evaluate(x_test,y_test)
        # my_net.train(False)
        # with torch.no_grad():
        #     x_test =  torch.DoubleTensor(x_test).cuda()
        #     answer = my_net(x_test)
        #     probs=answer.cpu().numpy()
        #     preds       = probs.argmax(axis = -1)
        #     acc=accuracy_score(y_test, preds)

        #     print(acc)
        #     results[i-1]=acc


    # print('mean accuracy:',np.mean(results))

Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py", line 354, in compute_output_shape
        raise ValueError(

    ValueEr

In [ ]:
layers.Conv2D?

In [ ]:
layers.BatchNormalization?

In [ ]:
layers.ELU?

In [43]:
layers.AveragePooling2D?

In [44]:
layers.Reshape?

In [49]:
layers.LogSoftmax?

Object `layers.LogSoftmax` not found.
